# Feature Preparation for Clustering Trips and Stations

In [1]:
import pandas as pd

In [2]:
trips_df = pd.read_pickle('../00_data/trips.pkl')
trips_df.head(2)

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,duration,start_latitude,...,sustenance_poi_start,public_transport_poi_start,education_poi_start,arts_and_culture_poi_start,sports_poi_start,sustenance_poi_end,public_transport_poi_end,education_poi_end,arts_and_culture_poi_end,sports_poi_end
0,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,05992,Walk-up,Main & 1st,Broadway & 9th,0 days 00:32:00,34.05194,...,80,28,2,4,0,67,51,0,4,4
1,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,3075,05860,Walk-up,Main & 1st,Broadway & 9th,0 days 00:30:00,34.05194,...,80,28,2,4,0,67,51,0,4,4


In [3]:
trips_hourly_df = pd.read_pickle('../00_data/trips_hourly_selected.pkl')
trips_hourly_df.head(2)

,starting_trips,ongoing_trips_prev,available_bikes,min_temp,hour,month,is_weekday,is_holiday
index,,,,,,,,
2019-01-01 01:00:00,8.0,2.0,870.0,15.6,1,1,True,True
2019-01-01 02:00:00,11.0,4.0,868.0,15.0,2,1,True,True


Let's merge the hourly data to our trip dataframe. To do that we have to have a column that has the same values as the hourly-data-index. 

In [4]:
trips_df["start_time_floored"]=trips_df['start_time'].dt.floor('H')
trips_df = trips_df.sort_values("start_time")
len_before_merge = len(trips_df)
trips_df = trips_df.merge(trips_hourly_df, left_on='start_time_floored', right_index=True)
trips_df.head(2)

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,duration,start_latitude,...,sports_poi_end,start_time_floored,starting_trips,ongoing_trips_prev,available_bikes,min_temp,hour,month,is_weekday,is_holiday
210759,2019-01-01 01:29:00,2019-01-01 02:49:00,4211,4211,12244,Walk-up,Pacific & North Venice,Pacific & North Venice,0 days 01:20:00,33.98493,...,0,2019-01-01 01:00:00,8.0,2.0,870.0,15.6,1,1,True,True
55231,2019-01-01 01:31:00,2019-01-01 01:35:00,3027,3046,05819,Walk-up,Spring & 3rd,2nd & Hill,0 days 00:04:00,34.04998,...,1,2019-01-01 01:00:00,8.0,2.0,870.0,15.6,1,1,True,True


Now we can compute how many rows we lost:

In [5]:
print(len_before_merge - len(trips_df))

14


Those 14 trips were started between 0:00 am and 1:00 am at the first of january 2019. For this hour there is no hourly
data created so it was dropped when merging.
Now we will edit some columns to our needs:
- set the duration in minutes
- set an end hour
- change boolean values to 1 and 0
- change user types to numbers


In [6]:
trips_df['duration'] = trips_df['duration'].dt.seconds/60
trips_df["hour_end"] = trips_df['end_time'].dt.hour
trips_df["is_weekday"] = trips_df["is_weekday"].apply(lambda x: int(x))
trips_df["is_holiday"] = trips_df["is_holiday"].apply(lambda x: int(x))
types = ['Walk-up', 'Monthly Pass', 'Annual Pass', 'One Day Pass', 'Flex Pass', 'Testing']
trips_df["user_type"] = trips_df["user_type"].apply(lambda x: types.index(x) if type(x) == str else None)

trips_df.head(2)

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,duration,start_latitude,...,start_time_floored,starting_trips,ongoing_trips_prev,available_bikes,min_temp,hour,month,is_weekday,is_holiday,hour_end
210759,2019-01-01 01:29:00,2019-01-01 02:49:00,4211,4211,12244,0,Pacific & North Venice,Pacific & North Venice,80.0,33.98493,...,2019-01-01 01:00:00,8.0,2.0,870.0,15.6,1,1,1,1,2
55231,2019-01-01 01:31:00,2019-01-01 01:35:00,3027,3046,05819,0,Spring & 3rd,2nd & Hill,4.0,34.04998,...,2019-01-01 01:00:00,8.0,2.0,870.0,15.6,1,1,1,1,1


Lastly let us take a look at the columns and which of them we will need. We drop the others and save the dataframe as a pickle.

In [7]:
trips_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225280 entries, 210759 to 130228
Data columns (total 37 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   start_time                  225280 non-null  datetime64[ns]
 1   end_time                    225280 non-null  datetime64[ns]
 2   start_station_id            225280 non-null  int64         
 3   end_station_id              225280 non-null  int64         
 4   bike_id                     225280 non-null  object        
 5   user_type                   225280 non-null  int64         
 6   start_station_name          225280 non-null  object        
 7   end_station_name            225280 non-null  object        
 8   duration                    225280 non-null  float64       
 9   start_latitude              225280 non-null  float64       
 10  start_longitude             225280 non-null  float64       
 11  end_latitude                225280

In [8]:
trips_with_hourly_df = trips_df.drop(columns=[
    'start_time','end_time',                    # start and end hour accurate enough
    'start_station_id', 'end_station_id',       # station id not important only where it is
    'start_station_name', 'end_station_name',   # station name not important only where it is
    'bike_id', 'start_hex', 'end_hex',          # bike_id unique and hex not for clustering
    'sustenance_poi_start', 'public_transport_poi_start', # POI's dependent on location
    'education_poi_start', 'arts_and_culture_poi_start',  # we want to cluster location via latitude and longitude
    'sports_poi_start', 'sustenance_poi_end',
    'public_transport_poi_end', 'education_poi_end',
    'arts_and_culture_poi_end', 'sports_poi_end',
    'start_time_floored',                       # start time in start_hour
    'starting_trips', 'ongoing_trips_prev',     # not important for single trip
    'available_bikes'                           # not important for single trip
   ])
trips_with_hourly_df = trips_with_hourly_df.rename(columns={"hour": "hour_start", "min_temp": "temp"})

In [9]:
trips_with_hourly_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225280 entries, 210759 to 130228
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   user_type        225280 non-null  int64  
 1   duration         225280 non-null  float64
 2   start_latitude   225280 non-null  float64
 3   start_longitude  225280 non-null  float64
 4   end_latitude     225280 non-null  float64
 5   end_longitude    225280 non-null  float64
 6   distance         225280 non-null  float64
 7   speed            225280 non-null  float64
 8   temp             225280 non-null  float64
 9   hour_start       225280 non-null  int64  
 10  month            225280 non-null  int64  
 11  is_weekday       225280 non-null  int64  
 12  is_holiday       225280 non-null  int64  
 13  hour_end         225280 non-null  int64  
dtypes: float64(8), int64(6)
memory usage: 25.8 MB


In [10]:
trips_with_hourly_df.to_pickle('../00_data/trips_with_hourly_features.pkl')

# Feature Preparation for clustering locations based on demand

We filter only demand related features from the trips dataframe and save them to trips_demand_df.

In [11]:
trips_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225280 entries, 210759 to 130228
Data columns (total 37 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   start_time                  225280 non-null  datetime64[ns]
 1   end_time                    225280 non-null  datetime64[ns]
 2   start_station_id            225280 non-null  int64         
 3   end_station_id              225280 non-null  int64         
 4   bike_id                     225280 non-null  object        
 5   user_type                   225280 non-null  int64         
 6   start_station_name          225280 non-null  object        
 7   end_station_name            225280 non-null  object        
 8   duration                    225280 non-null  float64       
 9   start_latitude              225280 non-null  float64       
 10  start_longitude             225280 non-null  float64       
 11  end_latitude                225280

In [12]:
trips_demand_df = trips_df[["start_time", "end_time", "start_station_id", "end_station_id", "hour", "month", "is_weekday", "is_holiday", "hour_end"]
]

In [13]:
trips_demand_df.head(2)

,start_time,end_time,start_station_id,end_station_id,hour,month,is_weekday,is_holiday,hour_end
210759,2019-01-01 01:29:00,2019-01-01 02:49:00,4211,4211,1,1,1,1,2
55231,2019-01-01 01:31:00,2019-01-01 01:35:00,3027,3046,1,1,1,1,1


Then we create different features from our demand related data:

- Demnad at different times in a day

In [14]:
# filter dataframe for different features
morning_df = trips_demand_df[(trips_demand_df["hour"] > 4) & (trips_demand_df["hour"] <= 10)]
noon_df = trips_demand_df[(trips_demand_df["hour"] > 10) & (trips_demand_df["hour"] <= 16)]
evening_df = trips_demand_df[(trips_demand_df["hour"] > 16) & (trips_demand_df["hour"] <= 22)]
night_df = trips_demand_df[(trips_demand_df["hour"] > 22) | (trips_demand_df["hour"] <= 4)]

morning_end_df = trips_demand_df[(trips_demand_df["hour_end"] > 4) & (trips_demand_df["hour_end"] <= 10)]
noon_end_df = trips_demand_df[(trips_demand_df["hour_end"] > 10) & (trips_demand_df["hour_end"] <= 16)]
evening_end_df = trips_demand_df[(trips_demand_df["hour_end"] > 16) & (trips_demand_df["hour_end"] <= 22)]
night_end_df = trips_demand_df[(trips_demand_df["hour_end"] > 22) | (trips_demand_df["hour_end"] <= 4)]

# save time new features to stations
start_all = trips_demand_df.groupby("start_station_id").size()
stations_new_df = pd.DataFrame({"start_all": start_all})
stations_new_df["end_all"] = trips_demand_df.groupby("end_station_id").size()
stations_new_df["fluctuation"] = stations_new_df["end_all"] - stations_new_df["start_all"]

stations_new_df["start_morning"] = morning_df.groupby("start_station_id").size()
stations_new_df["start_noon"] = noon_df.groupby("start_station_id").size()
stations_new_df["start_evening"] = evening_df.groupby("start_station_id").size()
stations_new_df["start_night"] = night_df.groupby("start_station_id").size()

stations_new_df["end_morning"] = morning_end_df.groupby("end_station_id").size()
stations_new_df["end_noon"] = noon_end_df.groupby("end_station_id").size()
stations_new_df["end_evening"] = evening_end_df.groupby("end_station_id").size()
stations_new_df["end_night"] = night_end_df.groupby("end_station_id").size()

- Demnad in different seasons

In [15]:
# filter dataframe for different features
summer_df = trips_demand_df[(trips_demand_df["month"] > 3) & (trips_demand_df["hour"] <= 9)]
winter_df = trips_demand_df[(trips_demand_df["month"] > 9) | (trips_demand_df["hour"] <= 3)]

# save time new features to stations
stations_new_df["start_summer"] = summer_df.groupby("start_station_id").size()
stations_new_df["end_summer"] = summer_df.groupby("end_station_id").size()
stations_new_df["start_winter"] = winter_df.groupby("start_station_id").size()
stations_new_df["end_winter"] = winter_df.groupby("end_station_id").size()

- Demnad on weekends and holidays

In [16]:
# filter dataframe for different features
holiday_df = trips_demand_df[trips_demand_df["is_holiday"] == 1]
weekday_df = trips_demand_df[trips_demand_df["is_weekday"] == 1]
weekend_df = trips_demand_df[trips_demand_df["is_weekday"] == 0]

# save time new features to stations
stations_new_df["start_holiday"] = holiday_df.groupby("start_station_id").size()
stations_new_df["end_holiday"] = holiday_df.groupby("end_station_id").size()
stations_new_df["start_week"] = weekday_df.groupby("start_station_id").size()
stations_new_df["end_week"] = weekday_df.groupby("end_station_id").size()
stations_new_df["start_weekend"] = weekend_df.groupby("start_station_id").size()
stations_new_df["end_weekend"] = weekend_df.groupby("end_station_id").size()

Lastly we fill Null values with the numeric integer 0 and save the location data in a pickle file.

In [17]:
stations_new_df = stations_new_df.fillna(0)
stations_new_df

,start_all,end_all,fluctuation,start_morning,start_noon,start_evening,start_night,end_morning,end_noon,end_evening,...,start_summer,end_summer,start_winter,end_winter,start_holiday,end_holiday,start_week,end_week,start_weekend,end_weekend
start_station_id,,,,,,,,,,,,,,,,,,,,,
3005,13337,14189,852,2369.0,4714.0,5671.0,583.0,3652.0,5105.0,4784,...,1665.0,2589.0,3971.0,4155.0,542.0,601.0,10404,11306,2933.0,2883.0
3006,6304,6569,265,849.0,2475.0,2632.0,348.0,689.0,2755.0,2816,...,579.0,474.0,1660.0,1677.0,295.0,322.0,4586,4765,1718.0,1804.0
3007,4930,4332,-598,491.0,2315.0,2039.0,85.0,1310.0,1926.0,1006,...,272.0,977.0,1391.0,1321.0,228.0,180.0,4046,3569,884.0,763.0
3008,3566,3434,-132,398.0,1395.0,1573.0,200.0,762.0,1271.0,1237,...,302.0,541.0,1075.0,1036.0,169.0,147.0,2684,2545,882.0,889.0
3010,793,792,-1,36.0,376.0,368.0,13.0,196.0,456.0,125,...,25.0,141.0,224.0,204.0,42.0,45.0,652,592,141.0,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4482,29,33,4,2.0,15.0,12.0,0.0,2.0,20.0,10,...,0.0,2.0,29.0,33.0,1.0,2.0,20,24,9.0,9.0
4483,43,34,-9,5.0,28.0,6.0,4.0,2.0,22.0,6,...,6.0,4.0,43.0,34.0,4.0,2.0,30,23,13.0,11.0
4490,1,1,0,0.0,0.0,1.0,0.0,0.0,0.0,1,...,0.0,0.0,1.0,1.0,1.0,1.0,1,1,0.0,0.0


In [18]:
stations_new_df.to_pickle('../00_data/station_demand.pkl')